In [2]:
# (CPU, DataSize, Y=Time)
import pandas as pd

In [3]:
df = pd.read_csv("testing_28July_bowtie.csv", sep=",") #data from 22July2016 
# copy the 'dup' folder to ~/ProfilingData/.
# remove *Output* files 
# run "python extract.py dup/" to generate testing.csv

In [4]:
len(df)

64

In [5]:
df.head(n=1)

,ppn,mem,seqlen,trial,AvgRSS,AvgSharedTxt,AvgStack,AvgTotal,AvgUnsharedData,CommandTimed,...,PageSize,PercentCPU,SignalsDeliv,SocketMsgRecv,SocketMsgSent,Swaps,SystemTime,UserTime,VolCtxtSwitch,WallTime
0,11,64G,10M,1,0,0,0,0,0,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 1...,...,4096,883%,0,0,0,0,27.49,814.03,823,1:35.29


In [6]:
#find which columns are completely empty
a=len(df[df['AvgRSS'] > 0])
b=len(df[df['AvgSharedTxt'] > 0])
c=len(df[df['AvgStack'] > 0])
d=len(df[df['AvgTotal'] > 0])
e=len(df[df['AvgUnsharedData'] > 0])

print(a,b,c,d,e)

0 0 0 0 0


In [7]:
header = list(df.columns)

In [8]:
i = 1
a = 0
emptyColumns = []
for h in header:
    try:
        a=len(df[df[h] > 0])
        print(a,  h)
        if(a==0): #nothing is greater than zero for this column
            emptyColumns.append(h)
    except:
        print('......................'+ h +' not a scalar')
    i=i+1

64 ppn
......................mem not a scalar
......................seqlen not a scalar
64 trial
0 AvgRSS
0 AvgSharedTxt
0 AvgStack
0 AvgTotal
0 AvgUnsharedData
......................CommandTimed not a scalar
0 ExitStatus
64 FileSysIn
64 FileSysOut
64 InvolCtxtSwitch
32 MajorFaults
64 MaxRSS
64 MinorFaults
64 PageSize
......................PercentCPU not a scalar
0 SignalsDeliv
0 SocketMsgRecv
0 SocketMsgSent
0 Swaps
64 SystemTime
64 UserTime
64 VolCtxtSwitch
......................WallTime not a scalar


In [9]:
emptyColumns.append('trial')
emptyColumns.append('PageSize')
#emptyColumns.append('CommandTimed')
emptyColumns.append('WallTime')

In [10]:
for c in emptyColumns:
    del df[c]

In [22]:
pd.options.display.max_colwidth=500

In [42]:
df.head(4)[["CommandTimed"]]

,CommandTimed
0,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-1 ./outputDir/human-hit-1
1,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-2 ./outputDir/human-hit-2
2,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-1 ./outputDir/human-hit-1
3,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-2 ./outputDir/human-hit-2


In [29]:
commandmap = {'./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-1 ./outputDir/human-hit-1':
             'fltd-1',
             './NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-2 ./outputDir/human-hit-2':
             'fltd-2'}

In [30]:
ft = df.copy(deep=True)

In [31]:
ft['oneORtwo'] = ft['CommandTimed'].map(commandmap)

In [38]:
ft.head(4)[['oneORtwo', 'CommandTimed']]

,oneORtwo,CommandTimed
0,fltd-1,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-1 ./outputDir/human-hit-1
1,fltd-2,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-2 ./outputDir/human-hit-2
2,fltd-1,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-1 ./outputDir/human-hit-1
3,fltd-2,./NGS-ann-program/BIN/bowtie -f -k 1 -v 2 -p 16 ./NodeRefDir/human/human ./data/fltd-2 ./outputDir/human-hit-2


In [43]:
delthis = list(set(ft.columns.values) - set(['ppn','seqlen','SystemTime','UserTime','oneORtwo']))
delthis

[]

In [44]:
for c in delthis:
    del ft[c]

In [45]:
#DROP the lines which are NULL
#df = df[df.CommandTimed.notnull()]
len(df)

64

In [50]:
ft['seqFile']=ft['seqlen']+ft['oneORtwo']
ft.head(5)

KeyError: 'seqlen'

In [56]:
whatisthesize = {'5Mfltd-1': .27, '10Mfltd-1': .56,'15Mfltd-1': .85,'20Mfltd-1': 1.2,'25Mfltd-1': 1.5,
                 '30Mfltd-1': 1.7,'35Mfltd-1': 2.0,'40Mfltd-1': 2.3, 
                 '5Mfltd-2': .27, '10Mfltd-2': .56,'15Mfltd-2': .85,'20Mfltd-2': 1.2,'25Mfltd-2': 1.5,
                 '30Mfltd-2': 1.7,'35Mfltd-2': 2.0,'40Mfltd-2': 2.3}

In [54]:
ft['sizeGB'] = ft['seqFile'].map(whatisthesize)
ft.head(5)

,ppn,SystemTime,UserTime,seqFile,sizeGB
0,11,27.49,814.03,10Mfltd-1,0.56
1,11,27.21,801.46,10Mfltd-2,0.56
2,11,27.72,1225.76,15Mfltd-1,0.85
3,11,27.28,1217.28,15Mfltd-2,0.85
4,11,27.69,1647.21,20Mfltd-1,1.20


In [55]:
del ft['seqFile']
del ft['seqlen']
del ft['oneORtwo']
ft.head(2)

KeyError: 'seqlen'

In [94]:
ft['Y_time']=ft['SystemTime']+ft['UserTime']
del ft['SystemTime']
del ft['UserTime']


KeyError: 'SystemTime'

In [96]:
ft.head(20)

,ppn,sizeGB,Y_time
0,11,0.56,841.52
1,11,0.56,828.67
2,11,0.85,1253.48
3,11,0.85,1244.56
4,11,1.20,1674.90
5,11,1.20,1665.97
6,11,1.50,2076.99
7,11,1.50,2068.35
8,11,1.70,2473.27
9,11,1.70,2459.98


In [97]:
from sklearn.ensemble.forest import RandomForestRegressor
import numpy as  np

In [100]:
ft.columns[0:2]

Index(['ppn', 'sizeGB'], dtype='object')

In [101]:
ft.columns[2]

'Y_time'

In [283]:
model = RandomForestRegressor(n_estimators=1500, n_jobs=-1)

In [328]:
np.random.seed()
ft['is_train']=np.random.uniform(0,1,len(ft)) <= .85
train, test = ft[ft['is_train']==True], ft[ft['is_train']==False]
del ft['is_train']
len(train)*1.0 / (len(test)+len(train))

0.8125

In [329]:
features = ft.columns[0:2]
target = ft.columns[2]

X_train = train[features]
Y_train = train[target]

X_test = test[features]
Y_test = test[target].as_matrix()

In [330]:
model.fit(X_train, Y_train.values)
Y_prediction = model.predict(X_test)

In [348]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_test, Y_prediction))
print(rmse)

398.189685558


In [349]:
while(rmse > 50):
    np.random.seed()
    ft['is_train']=np.random.uniform(0,1,len(ft)) <= .85
    train, test = ft[ft['is_train']==True], ft[ft['is_train']==False]
    del ft['is_train']
    len(train)*1.0 / (len(test)+len(train))
    features = ft.columns[0:2]
    target = ft.columns[2]
    X_train = train[features]
    Y_train = train[target]
    X_test = test[features]
    Y_test = test[target].as_matrix()
    model.fit(X_train, Y_train.values)
    Y_prediction = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(Y_test, Y_prediction))


In [354]:
rmse

21.491244822893599

In [350]:
Y_test #Print Y values of test dataset

array([ 2872.17,  1273.04,   426.53,   435.69,  1706.37,  2513.88])

In [351]:
Y_prediction #Print Y predicted values of test dataset

array([ 2914.02754087,  1241.79475478,   430.02694333,   430.75347452,
        1707.98127715,  2515.81091605])

In [352]:
X_test = test[features]
Y_test = test[target]

model.score(X_test, Y_test) #which score is this ?
# coefficient of determination R^2 of the prediction

0.99947527843967443

In [353]:
ft[target].mean(), ft[target].std() * 2

(2030.3582812499994, 2415.6120627747564)

In [336]:
model = RandomForestRegressor(n_estimators=1500, n_jobs=-1)

In [345]:
from sklearn import cross_validation

#returns -ve of MSE
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=20,scoring='mean_squared_error')

In [346]:
#convert MSE to RMSE
rmse = np.sqrt(-scores)
rmse

array([  578.965681  ,    36.28181401,    95.90149233,    39.19571574,
          96.92974189,    18.89826218,   504.34400447,  2717.52987086,
        2042.0387125 ,   311.48439441,    65.06462082,    23.75005604,
         108.20934114,   504.10240749,   195.18751721,   227.59165339,
          20.18759262,    34.44629866,    76.43154936,    33.08936714])

In [342]:
print("Accuracy: %0.2f (+/- %0.2f)" % (rmse.mean(), rmse.std() * 2))

Accuracy: 614.07 (+/- 1388.90)


In [343]:
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=5,scoring='r2')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.35 (+/- 1.13)


In [344]:
scores

array([ 0.82666571, -0.1608329 ,  0.53882929, -0.46552011,  0.99789867])

## Gradient Boosting Regressor

In [162]:
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

In [201]:
params_ada = {'n_estimators': 5000,       'learning_rate': 0.001, 'loss': 'linear'}

#clf = ensemble.GradientBoostingRegressor(**params)

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

clf = RandomForestRegressor(n_estimators=500, n_jobs=-1)

clf.fit(X_train, Y_train.as_matrix())

mse = mean_squared_error(Y_test, clf.predict(X_test))

print("MSE: %.4f" % np.sqrt(mse))


MSE: 175.8904


In [202]:
from sklearn import cross_validation
kfold = cross_validation.KFold(len(ft), n_folds=3)
scores = cross_validation.cross_val_score(clf, ft[features], ft[target].as_matrix(), cv=3,scoring='mean_squared_error')
scores

array([  -62253.07373555, -1066673.12324349,  -123392.07858259])

In [203]:
from sklearn.metrics import mean_squared_error
rms = np.sqrt(mean_squared_error(Y_test, clf.predict(X_test)))
print(rms)

175.890394282


In [195]:

feature_importance = clf.feature_importances_

feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)

pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X_train.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

AttributeError: 'KNeighborsRegressor' object has no attribute 'feature_importances_'